## Q3: Tossing a brick on ice (40 pts)

In [ ]:
using Pkg;
const root = dirname(@__DIR__)
Pkg.activate(root);
Pkg.instantiate()

In [ ]:
using LinearAlgebra
using ForwardDiff
using Plots
using OrdinaryDiffEq
using Ipopt
using MathOptInterface
const MOI = MathOptInterface
using Test
const FD = ForwardDiff
using JLD2, FileIO
const jld2path = joinpath(root,"test");
include(joinpath(jld2path,"test_functions.jl"));

In [ ]:
#Brick Parameters
const g = 9.81
const m = 1
const ℓ = 1
const J = Diagonal([0.5; 1; 1.5]);

In [ ]:
#Some standard functions for dealing with rotation matrices and quaternions from the class notes
#(No need to edit these)

function hat(ω)
    return [0 -ω[3] ω[2];
            ω[3] 0 -ω[1];
            -ω[2] ω[1] 0]
end

function L(Q)
    [Q[1] -Q[2:4]'; Q[2:4] Q[1]*I + hat(Q[2:4])]
end

function R(Q)
    [Q[1] -Q[2:4]'; Q[2:4] Q[1]*I - hat(Q[2:4])]
end

const H = [zeros(1,3); I];

const T = Diagonal([1.0; -1; -1; -1])

function G(Q)
    return L(Q)*H
end

function Ḡ(q)
    Q = q[4:7]
    return [I zeros(3,3); zeros(4,3) G(Q)]
end
function ρ(ϕ)
    # convert from ϕ to a quaternion 
    # read more: https://roboticexplorationlab.org/papers/planning_with_attitude.pdf
    return (1/sqrt(1 + dot(ϕ,ϕ)))*[1;ϕ]
end

## 3A: Newton-Euler dynamics (10 pts)

In [ ]:
#Simulation Parameters
const h = 0.05 #20 Hz
const Tf = 5.0 #final time (sec)
const Thist = Array(0:h:Tf)
const N = length(Thist);

In [ ]:
#Initial Conditions
using Random
Random.seed!(1234)
const r0 = [0; 0; 3.0]
const v0 = [1.0; 0; 0.0]
const ϕ0 = 0.2*randn(3)
const Q0 = [sqrt(1.0-ϕ0'*ϕ0); ϕ0] #Random initial attitude
const ω0 = [1e-2*randn(); 4; 1e-2*randn()] #spin about the unstable intermediate axis of inertia
const x0 = [r0; Q0; v0; ω0];

In [ ]:
#Reference Solution

function f(x,p,t)
    # Don't worry about p and t, they are not used in this function. They're just for the DifferentialEquations.jl interface.
    
    #Unpack state vector
    r = x[1:3]   # position in world frame 
    Q = x[4:7]   # ^N Q ^B (quaternion)
    v = x[8:10]  # velocity in world frame
    ω = x[11:13] # ^N ω ^B (angular velocity)
    
    # TODO: Implement Newton-Euler dynamics
    ẋ = zeros(eltype(x),length(x))
    
    return ẋ
end

In [ ]:
@testset "dynamics" begin 
    r = [1;2;4.3]
    Q = normalize([.1;.3;-.2;.3])
    v = [-1;.3;4]
    ω = [.2;.3;-.1]
    x = [r;Q;v;ω]
    ẋ1 = f(x,0,0)
    test_vec(ẋ1,[-1.0, 0.3, 4.0, 0.03127716210856121, -0.05212860351426868, 0.12510864843424485, 0.12510864843424485, 0.0, 0.0, -9.81, 0.03, -0.020000000000000004, -0.02] )
end

In [ ]:
tspan = (0.0,Tf)
prob = ODEProblem(f,x0,tspan)
sol = solve(prob,Tsit5(),reltol=1e-8,abstol=1e-8);

In [ ]:
#You should see the object falling in the z-direction while maintaining constant velocity in the x-direction
plot(sol, vars=(0,1),label = "r_x")
plot!(sol, vars=(0,2),label = "r_y")
plot!(sol, vars=(0,3),label = "r_z")
xlabel!("time (s)")
ylabel!("m")
title!("position history")

In [ ]:
#You should see a characteristic flipping of the rotation axis due to spinning about the unstable intermediate axis
plot(sol, vars=(0,11),label = "ωₓ")
plot!(sol, vars=(0,12),label = "ω_y")
plot!(sol, vars=(0,13),label = "ω_z")
xlabel!("time (s)")
ylabel!("rad/s")
title!("angular velocity history")

## 3B: Variational Integrator (15 pts)

In [ ]:
function DLT1(q1,q2)
    r1 = q1[1:3]
    Q1 = q1[4:7]
    r2 = q2[1:3]
    Q2 = q2[4:7]
    
    # TODO: output the right discrete Legendre transform for a free rigid body
    
    return NaN
end


In [ ]:
@testset "DLT1" begin 
    r1 = [1;2;4.3]
    Q1 = normalize([.1;.3;-.2;.3])
    q1 = [r1;Q1]
    r2 = [1.3;-2;4.6]
    Q2 = normalize([.14;.2;-.1;.4])
    q2 = [r2;Q2]   
    D1 = DLT1(q1,q2)
    test_vec(D1, [-6.000000000000001, 80.0, -6.245249999999997, -1.9148613846386908, -12.898045750643837, 2.090592334494776])
end

In [ ]:
function DLT2(q1,q2)
    r1 = q1[1:3]
    Q1 = q1[4:7]
    r2 = q2[1:3]
    Q2 = q2[4:7]
    
    # TODO: output the left discrete Legendre transform for a free rigid body
    
    return NaN
end

In [ ]:
@testset "DLT2" begin 
    r1 = [1;2;4.3]
    Q1 = normalize([.1;.3;-.2;.3])
    q1 = [r1;Q1]
    r2 = [1.3;-2;4.6]
    Q2 = normalize([.14;.2;-.1;.4])
    q2 = [r2;Q2]   
    D2 = DLT2(q1,q2)
    test_vec(D2, [6.000000000000001, -80.0, 5.754749999999996, 2.6238448719890917, 12.389031964853807, -3.744887138312375])
end

In [ ]:
function DEL(q1,q2,q3)   
    # TODO: output the discrete Euler-Lagrange equation for a free rigid body
    
    return NaN
end

In [ ]:
@testset "DLT2" begin 
    r1 = [1;2;4.3]
    Q1 = normalize([.1;.3;-.2;.3])
    q1 = [r1;Q1]
    r2 = [1.3;-2;4.6]
    Q2 = normalize([.14;.2;-.1;.4])
    q2 = [r2;Q2]   
    r3 = [2.3;-1;4.32]
    Q3 = normalize([.17;.23;-.112;.49])
    q3 = [r3;Q3] 
    D = DEL(q1,q2,q3)
    test_vec(D,[-13.999999999999996, -100.0, 11.109499999999983, 2.4506935890672716, 11.332809139030706, -3.749462832864676])
end

In [ ]:
#First let's first simulate the rigid body with no contact using our midpoint variational integrator to make sure it looks reasonable

#Initial conditions
qhist = zeros(7,N)
qhist[:,1] .= [r0; Q0]
qhist[:,2] .= [r0 + h*v0; Q0 + h*0.5*L(Q0)*H*ω0]
qhist[4:7,2] .= qhist[4:7,2]/norm(qhist[4:7,2])

# TODO: simulate using the midpoint variational integrator
for k = 2:(N-1)
    qhist[:,k+1] .= qhist[:,k] #initial guess
    e = DEL(qhist[:,k-1], qhist[:,k], qhist[:,k+1]) #DEL residual
    
    # TODO: Implement Newton's method to solve for q_k+1. Remember to properly handle the quaternion.
    # this means that we will use the Ḡ function we defined earlier to modify the jacobian.
    # After you solve for the Newton step, apply the step additively to the position, and 
    # apply the step multiplicatively to the quaternion

    # hint: at each iteration we are solving for qhist[:,k+1] such that:
    # DEL(qhist[:,k-1], qhist[:,k], qhist[:,k+1]) = zeros()
    

    # test the residual is close to 0 (aka, Newton's method converged properly)
    @test norm(DEL(qhist[:,k-1], qhist[:,k], qhist[:,k+1])) < 1e-6
end

#Calculate midpoint velocity approximations for plotting
vhist = zeros(3,N-1)
ωhist = zeros(3,N-1)
for k = 1:(N-1)
    
    # TODO: Implement midpoint finite-difference approximation for translational + angular velocity
    # hint: you may assume that the quaternion ^B(k)Q^B(k+1) is [1;ωk*h/2] (where ^NQ^B(k) = qhist[4:7,k])
    
    
end

In [ ]:
@testset "midpoint velocities" begin 
    for k = 1:(N-1)
        @test isapprox(qhist[1:3,k+1], qhist[1:3,k] + vhist[:,k]*h,rtol = 1e-9)
        @test isapprox( H'*L(qhist[4:7,k])'*qhist[4:7,k+1],ωhist[:,k]*h/2,rtol = 1e-9)
    end
end 

In [ ]:
#Compare with reference solution. You should see similar behavior
plot(Thist, qhist[1,:],label = "r_x")
plot!(Thist, qhist[2,:],label = "r_y")
plot!(Thist, qhist[3,:],label = "r_z")
plot!(sol, vars=(0,1),label = "true r_x",color = :red,linestyle = :dash)
plot!(sol, vars=(0,2),label = "true r_y",color = :red,linestyle = :dash)
plot!(sol, vars=(0,3),label = "true r_z",color = :red,linestyle = :dash)
xlabel!("time (s)")
ylabel!("m")
title!("position history")

In [ ]:
@test norm(sol(Tf)[1:3] - qhist[1:3,end]) < 5

In [ ]:
#Compare with reference solution. You should see similar behavior (spin axis flip)
plot(Thist[1:end-1],ωhist[1,:],label = "ωₓ")
plot!(Thist[1:end-1],ωhist[2,:],label = "ω_y")
plot!(Thist[1:end-1],ωhist[3,:],label = "ω_z")
plot!(sol, vars=(0,11),label = "true ω_x",color = :red,linestyle = :dash)
plot!(sol, vars=(0,12),label = "true ω_y",color = :red,linestyle = :dash)
plot!(sol, vars=(0,13),label = "true ω_z",color = :red,linestyle = :dash)
xlabel!("time (s)")
ylabel!("rad/s")
title!("angular velocity history")

In [ ]:
@test norm(sol(Tf)[11:13] - ωhist[1:3,end]) < .5
@test norm(sol(Tf)[8:10] - vhist[1:3,end]) < 2

In [ ]:
#Visualize the falling brick

using MeshCat
using CoordinateTransformations
using Rotations
using GeometryBasics: HyperRectangle, Vec, Point, Mesh
using Colors: RGBA, RGB

vis = Visualizer()
render(vis)

In [ ]:
# you can use (ctrl + enter) to run this cell without shifting the window down 
delete!(vis)

# our box 
box = HyperRectangle(Vec(-0.5, -.5, -0.5), Vec(1, 1, 1))
setobject!(vis["box"],box)

for k = 1:N
    
    # set position and attitude
    attitude = LinearMap(UnitQuaternion(qhist[4:7,k]))
    position = Translation(qhist[1:3,k])
    settransform!(vis["box"], compose(position,attitude))
    sleep(h)
end

## 3C: Contact dynamics through optimization (15 pts)

We are now going to simulate the dynamics of the brick as it contacts the ice. To do this, we are going to solve the following optimization problem at every timestep. 
$$ \begin{align}
\min_{q_{k+1},\lambda_k,s_k} && \sum_{i=1}^8s_{k,i} \\
\text{st} && DEL(q_{k-1},q_k,q_{k+1}) &= 0 && \text{discrete Euler-Lagrange}\\ 
 && \|q_{k+1}[4:7]\|^2 - 1 &= 0 && \text{quaternion norm constraint}\\ 
 && d(q_{k+1}) &\geq 0 && \text{corners must stay above ground}\\ 
 && s_k - \text{diag}(\lambda_k)d(q_{k+1}) &\geq 0 && \text{collision constraint}\\
 && \lambda_k &\geq 0 \\ 
 && s_k &\geq 0
\end{align} $$

We will be using IPOPT to solve this, where IPOPT takes problems of the following form
$$ \begin{align}
\min_x && f(x) \\ 
\text{st} && c_L \leq c(x) &\leq c_U && \text{constraint function with constraint bounds} \\ 
&& x_L \leq x &\leq x_U && \text{primal bounds}
\end{align}$$

In [ ]:
#Boilerplate setup code to interface with IPOPT. Don't touch this except to turn on/off verbose IPOPT if you want (see comment below).

struct ProblemMOI <: MOI.AbstractNLPEvaluator
    n_nlp::Int
    m_nlp::Int
    idx_ineq
    obj_grad::Bool
    con_jac::Bool
    sparsity_jac
    sparsity_hess
    primal_bounds
    constraint_bounds
    hessian_lagrangian::Bool
end

function ProblemMOI(n_nlp,m_nlp;
        idx_ineq=(1:0),
        obj_grad=true,
        con_jac=true,
        sparsity_jac=sparsity_jacobian(n_nlp,m_nlp),
        sparsity_hess=sparsity_hessian(n_nlp,m_nlp),
        primal_bounds=primal_bounds(n_nlp),
        constraint_bounds=constraint_bounds(m_nlp,idx_ineq=idx_ineq),
        hessian_lagrangian=false)

    ProblemMOI(n_nlp,m_nlp,
        idx_ineq,
        obj_grad,
        con_jac,
        sparsity_jac,
        sparsity_hess,
        primal_bounds,
        constraint_bounds,
        hessian_lagrangian)
end

function constraint_bounds(m; idx_ineq=(1:0))
    c_l = zeros(m)

    c_u = zeros(m)
    c_u[idx_ineq] .= Inf
    
    return c_l, c_u
end

function row_col!(row,col,r,c)
    for cc in c
        for rr in r
            push!(row,convert(Int,rr))
            push!(col,convert(Int,cc))
        end
    end
    return row, col
end

function sparsity_jacobian(n,m)

    row = []
    col = []

    r = 1:m
    c = 1:n

    row_col!(row,col,r,c)

    return collect(zip(row,col))
end

function sparsity_hessian(n,m)

    row = []
    col = []

    r = 1:m
    c = 1:n

    row_col!(row,col,r,c)

    return collect(zip(row,col))
end

function MOI.eval_objective(prob::MOI.AbstractNLPEvaluator, x)
    objective(x)
end

function MOI.eval_objective_gradient(prob::MOI.AbstractNLPEvaluator, grad_f, x)
    ForwardDiff.gradient!(grad_f,objective,x)
    return nothing
end

function MOI.eval_constraint(prob::MOI.AbstractNLPEvaluator,g,x)
    constraint!(g,x)
    return nothing
end

function MOI.eval_constraint_jacobian(prob::MOI.AbstractNLPEvaluator, jac, x)
    ForwardDiff.jacobian!(reshape(jac,prob.m_nlp,prob.n_nlp), constraint!, zeros(prob.m_nlp), x)
    return nothing
end

function MOI.features_available(prob::MOI.AbstractNLPEvaluator)
    return [:Grad, :Jac]
end

MOI.initialize(prob::MOI.AbstractNLPEvaluator, features) = nothing
MOI.jacobian_structure(prob::MOI.AbstractNLPEvaluator) = prob.sparsity_jac

function ipopt_solve(x0,prob::MOI.AbstractNLPEvaluator;
        tol=1.0e-9,c_tol=1.0e-9,max_iter=10000)
    x_l, x_u = prob.primal_bounds
    c_l, c_u = prob.constraint_bounds

    nlp_bounds = MOI.NLPBoundsPair.(c_l,c_u)
    block_data = MOI.NLPBlockData(nlp_bounds,prob,true)

    solver = Ipopt.Optimizer()
    solver.options["max_iter"] = max_iter
    solver.options["tol"] = tol
    solver.options["constr_viol_tol"] = c_tol
    
    #Uncomment the following line to turn off verbose IPOPT output
    #solver.options["print_level"] = 0

    x = MOI.add_variables(solver,prob.n_nlp)

    for i = 1:prob.n_nlp
        xi = MOI.SingleVariable(x[i])
        MOI.add_constraint(solver, xi, MOI.LessThan(x_u[i]))
        MOI.add_constraint(solver, xi, MOI.GreaterThan(x_l[i]))
        MOI.set(solver, MOI.VariablePrimalStart(), x[i], x0[i])
    end

    # Solve the problem
    MOI.set(solver, MOI.NLPBlock(), block_data)
    MOI.set(solver, MOI.ObjectiveSense(), MOI.MIN_SENSE)
    MOI.optimize!(solver)

    # Get the solution
    res = MOI.get(solver, MOI.VariablePrimal(), x)

    return res
end

In [ ]:
function d(q)
    #TODO: Calculate the height (z coordinate) of the 8 corners of the brick in the world frame.
    #Assume it is a cube with sides of length ℓ
    #Output should be an 8×1 vector
    
    r = q[1:3] #position
    Q = q[4:7] #quaternion
    
    corners_z = zeros(eltype(q),8) # store the heights in here 
    """
    there are 8 corners on the cube, you should output the z coordinate in the world frame
    of of all 8 corners. Here is the order in which you should arrange the corner heights:
    
       x | y | z   (body frame) 
    1  +   +   +
    2  +   -   + 
    3  -   -   +       +------+.
    4  -   +   +       |`.    | `. 
    5  +   +   -       |  `+--+---+
    6  +   -   -       |   |  |   |
    7  -   -   -       +---+--+.  |  
    8  -   +   -        `. |    `.|
                          `+------+ 
                                   
    """
    
    return corners_z
end

function Dd(q)
    # Implement the kinematic Jacobian corresponding to d(q) above. Output should be a 8×6 matrix
    # hint: make sure you apply Ḡ appropriately since this is a jacobian with a quaternion involved
    
    return NaN
end

In [ ]:
@testset "height and kinematic jacobian" begin 
    r = [1;-2;3.21]
    Q = normalize([1;.2;.8;-.4])
    q = [r;Q]
    
    d1=d(q)
    Dd1 = Dd(q)
    
    test_vec(d1, [2.796956521739131, 2.927391304347826, 3.8839130434782607, 3.753478260869565, 2.536086956521739, 2.666521739130435, 3.623043478260869, 3.492608695652174])
    @test size(Dd1) == (8,6)
    test_vec(vec(Dd1),vec([0.0 0.0 1.0 0.39130434782608675 -1.2173913043478255 0.8260869565217388; 0.0 0.0 1.0 -0.13043478260869557 -1.2173913043478257 -1.0869565217391302; 0.0 0.0 1.0 -0.13043478260869565 -0.6956521739130433 -0.8260869565217388; 0.0 0.0 1.0 0.39130434782608675 -0.6956521739130431 1.08695652173913; 0.0 0.0 1.0 0.13043478260869565 0.6956521739130433 0.8260869565217388; 0.0 0.0 1.0 -0.39130434782608675 0.6956521739130431 -1.08695652173913; 0.0 0.0 1.0 -0.39130434782608675 1.2173913043478255 -0.8260869565217388; 0.0 0.0 1.0 0.13043478260869557 1.2173913043478257 1.0869565217391302]))
end 

In [ ]:
function DEL(q1,q2,q3,λ)    
    # TODO: mplement the constrained discrete Euler-Lagrange equation that accounts for the collision constraints and corresponding normal forces λ
    
    return NaN
end

In [ ]:
@testset "DEL" begin 
    r1 = [1;2;4.3]
    Q1 = normalize([.1;.3;-.2;.3])
    q1 = [r1;Q1]
    r2 = [1.3;-2;4.6]
    Q2 = normalize([.14;.2;-.1;.4])
    q2 = [r2;Q2] 
    r3 = [-1.8;2.2;1.7]
    Q3 = normalize([.34;-.2;.17;.04])
    q3 = [r3;Q3]
    λ = [1;-2;3;-4;5;-6;3;.2]
    
    D1 = DEL(q1,q2,q3,λ)
    test_vec(D1, [68.0, -164.0, 63.51949999999998, 12.649114939597263, 14.67908034467283, -0.9187419089856679])
end

In [ ]:
#Objective and constraint functions for IPOPT

function objective(z)
    qn = z[1:7]
    λ = z[8:15]
    s = z[16:23]
    
    return sum(s) #Minimize slacks associated with complementarity conditions
end

function constraint!(c,z)
    qn = z[1:7]
    λ = z[8:15]
    s = z[16:23]
    
    # TODO: update c to include the following constraints:
    # nonlinear DEL equation                (6 equality constraint)
    # quaternion norm squared - 1 = 0       (1 equality constraint)
    # corners must stay above the ground    (8 inequality constraint)
    # collision constraints                 (8 inequality constraint)
    
    # hint: you can edit in place by doing c .= instead of c = 
    
    # hint: for the DEL equation, we have access to q_hist[:,k-1] and q_hist[:,k] since 
    # they are global variables, and we are solving for the q_hist[:,k+1] (this is our variable qn)
    
    return nothing
end

#Specify the indicies of c (constraint output) that should be non-negative.
#The rest will be treated as equality constraints.
#This can vary depending on how you stacked up c above.

nonnegative_constraint_indices = (0:0) # update this

function primal_bounds(n)
    #Enforce simple bound constraints on the decision variables (e.g. positivity) here
    # x_l ≤ [q; λ; s] ≤ x_u
    
    # TODO: update these 
    x_l = ones(n)
    x_u = ones(n)

    return x_l, x_u
end

In [ ]:
#Solve with IPOPT
n_nlp = 23
m_nlp = 23
nlp_prob = ProblemMOI(n_nlp,m_nlp, idx_ineq=nonnegative_constraint_indices);

#Initial conditions 
qhist = zeros(7,N)
qhist[:,1] .= [r0; Q0]
qhist[:,2] .= [r0 + h*v0; Q0 + h*0.5*L(Q0)*H*ω0]
qhist[4:7,2] .= qhist[4:7,2]/norm(qhist[4:7,2])

λhist = zeros(8,N-1)
shist = zeros(8,N-1)
k = 0

for kk = 2:(N-1)
    k = kk
    z_guess = [qhist[:,k]; zeros(8); ones(8)]
    z_sol = ipopt_solve(z_guess,nlp_prob);
    qhist[:,k+1] .= z_sol[1:7]
    λhist[:,k] .= z_sol[8:15]
    shist[:,k] .= z_sol[16:23]
end

In [ ]:
#Verify that positions look reasonable and height remains above the ground
plot(qhist[1,:],label = "rx")
plot!(qhist[2,:],label = "ry")
plot!(qhist[3,:],label = "rz")
title!("brick position")
ylabel!("meters")
xlabel!("timestep")

In [ ]:
min_height = minimum([minimum(d(qhist[:,i])) for i = 1:size(qhist,2)])
@test min_height > -1e-3

In [ ]:
#Plot contact forces. You should see zeros when the brick is in the air, followed by big spikes around impacts,
#followed by steady-state values that should add up to m*g
plot(λhist',label = "")
title!("contact forces (λ)")
xlabel!("time step")
ylabel!("λ")

In [ ]:
@test minimum(vec(λhist))  > -1e-2
@test minimum(vec(shist))  > -1e-2
@test norm(vec(shist),Inf) <  1e-2

In [ ]:
λsum = [sum(abs.(λhist[:,i])) for i = 1:N-1]
plot(λsum, label = "total contact force")
plot!([1,N],[m*g,m*g],linestyle = :dash, color = :red,label = "force of gravity")
xlabel!("timestep")
ylabel!("force (N)")
title!("contact force and gravitational force")

In [ ]:
# test that we equalize to gravity
@test isapprox(λsum[end-5],m*g,atol = 1e-3)

In [ ]:
#Visualize
vis = Visualizer()
render(vis)

In [ ]:
# you can use (ctrl + enter) to run this cell without moving the page around
#Run visualization
delete!(vis)

# our box 
box = HyperRectangle(Vec(-0.5, -.5, -0.5), Vec(1, 1, 1))
setobject!(vis["box"],box)

# position history 
for k = 1:N
    
    # set position and attitude
    attitude = LinearMap(UnitQuaternion(qhist[4:7,k]))
    position = Translation(qhist[1:3,k])

    settransform!(vis["box"], compose(position,attitude))
    sleep(h)
end